# Maximum Variance Frontier (MVF) Calculation

This function calculates the Maximum Variance Frontier (MVF) using a pre-trained model and financial data.
It estimates expected returns, volatilities, and the Sharpe ratio for different cryptocurrencies and suggests an allocation based on the Sharpe ratio.


In [ ]:
import pickle
import pandas as pd
import numpy as np

def calculate_mvf():
    try:
        # Load the model and data
        model = pickle.load(open("return_model.pkl", "rb"))
        df = pd.read_csv("data/processed/features_data.csv")

        # Check for required columns
        if 'Close' not in df.columns or 'Crypto' not in df.columns:
            raise ValueError("The dataset does not contain the necessary columns.")

        # Compute returns per cryptocurrency
        df['Return'] = df.groupby('Crypto')['Close'].pct_change()
        df = df.dropna()

        features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Volatility', 'Amplitude', 
                    'Price_change', 'Volatility_ratio', 'High_low_spread', 'SMA_7', 'SMA_50',
                    'EMA_10', 'EMA_50', 'RoC', 'BB_upper', 'BB_lower', 'ATR', 'RSI', 'VWAP', 'OBV']

        cryptos = df['Crypto'].unique()
        expected_returns = {}
        volatilities = {}
        sharpe_ratios = {}

        risk_free_rate = 0.02 / 252  # Annualized risk-free rate divided by 252 trading days

        # Compute metrics for each cryptocurrency
        for crypto in cryptos:
            crypto_df = df[df['Crypto'] == crypto]
            X = crypto_df[features].values  # Ensure the same features are used
            predicted_returns = model.predict(X)

            mean_return = predicted_returns.mean()
            volatility = predicted_returns.std()
            sharpe_ratio = (mean_return - risk_free_rate) / volatility if volatility != 0 else 0

            expected_returns[crypto] = mean_return
            volatilities[crypto] = volatility
            sharpe_ratios[crypto] = sharpe_ratio

        # Allocation based on Sharpe Ratio
        total_sharpe = sum(sharpe_ratios.values())
        allocation = {crypto: sharpe_ratios[crypto] / total_sharpe for crypto in cryptos} if total_sharpe != 0 else {crypto: 1/len(cryptos) for crypto in cryptos}

        return {
            "message": "MVF calculation completed successfully.",
            "expected_return": np.mean(list(expected_returns.values())),
            "volatility": np.mean(list(volatilities.values())),
            "sharpe_ratio": np.mean(list(sharpe_ratios.values())),
            "allocation": allocation
        }
    except Exception as e:
        raise ValueError(f"Error in MVF calculation: {str(e)}")


result = calculate_mvf()
print(result)